In [1]:
#importing libraries
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999

from math import sqrt
import lightgbm as lgb
#from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
#from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score,precision_score

In [2]:
model_full = pd.read_csv("C:/Users/affiqazrin/Desktop/mmlspark/Data_FinalProject_READY4.csv")

response_name = ["indexedDeposit"]

feature_names = ["duration",
                 "campaign",
                 "pdays",
                 "previous",
                 "age",
                     
                 "indexedJob",
                 "indexedMarital",
                 "indexedEducation",
                 
                 "indexedHousing",
                 "indexedLoan",
                 "indexedContact",
                 
                 "indexedDay",
                 "indexedMonth",
                 "indexedPoutcome"]

n_features = len(feature_names)
mask = feature_names + response_name

model = model_full[mask]
print('Model dataset:\n', model.head(5))
# print('\nDescription of model dataset:\n', model[feature_names].describe(include='all'))
    
X = model[feature_names].values
y = model[response_name].values.ravel()
    
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.3,
                                                    random_state = 0)

print('Size of resampled data:')
print(' train shape... ', X_train.shape, y_train.shape)
print(' test shape.... ', X_test.shape, y_test.shape)

Model dataset:
    duration  campaign  pdays  previous  age  indexedJob  indexedMarital  \
0       136         1    999         0   19          10               1   
1       338         4    999         0   19          10               1   
2       213         3    999         0   19          10               1   
3      1161         5    999         0   19          10               1   
4       680         1    999         0   20           6               1   

   indexedEducation  indexedHousing  indexedLoan  indexedContact  indexedDay  \
0                 5               1            0               0           3   
1                 1               0            0               0           3   
2                 6               2            2               0           1   
3                 6               0            0               1           4   
4                 1               1            1               1           2   

   indexedMonth  indexedPoutcome  indexedDeposit  
0

In [3]:
#converting the dataset into proper LGB format 
d_train=lgb.Dataset(X_train, label=y_train)

In [4]:
#Specifying the parameters
params={}
params['learning_rate']=0.03
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='binary' #Binary target feature
params['metric']='binary_logloss' #metric for binary classification
params['max_depth']=10

In [5]:
#train the model 
clf=lgb.train(params,d_train,100) #train the model on 100 epocs

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 976, number of negative: 7614
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000877 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 431
[LightGBM] [Info] Number of data points in the train set: 8590, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.113620 -> initscore=-2.054281
[LightGBM] [Info] Start training from score -2.054281


In [6]:
#prediction on the test set
y_pred=clf.predict(X_test)

In [7]:
#rounding the values
y_pred=y_pred.round(0)

#converting from float to integer
y_pred=y_pred.astype(int)

In [8]:
print("ROC-AUC: ", roc_auc_score(y_pred,y_test))
print("RMSE: ", sqrt(mean_squared_error(y_pred,y_test)))
print("Precision Score: ", precision_score(y_pred,y_test))

ROC-AUC:  0.7979271043726883
RMSE:  0.30163415145730205
Precision Score:  0.4211764705882353
